In [8]:
import sys
sys.path.append("../../") 
sys.path.append("../IKRequests/") 
sys.path.append("./") 

import numpy as np
import pandas as pd
import re
import requests
import json

import utils as ut
import fdb_connect as db
import connect_ap as ap
from datetime import datetime, timedelta
from time import sleep

In [2]:
print('Connect to firebird')
conn = db.FirebirdConnection()

Connect to firebird


In [3]:
par_prot = {
    'year': 2018, 
    'month': 9
}

In [4]:
def kv_to_table(df_ik):
    """
    Convert table from key-value to shallow format
    """
    prots_lst = list(set(df_ik['PROTOCOLID']))
    res_list = []
    for tc in prots_lst:
        new_dict={}
        df_tmp = df_ik[df_ik['PROTOCOLID']==tc]
        suffixes = ['', '_MINNORM', '_MAXNORM']
        for idx, col_name in enumerate(('VALUETEXT', 'MINNORMVALUE', 'MAXNORMVALUE')):
            tmp_dict = dict(zip(df_tmp['F25_NAMEPARAMS'].tolist(), df_tmp[col_name].tolist()))
            tmp_dict = {i+suffixes[idx]: tmp_dict[i] for i in tmp_dict.keys()}
            tmp_dict = {re.sub("^\s+|\n|\r|\s+$",'',i): tmp_dict[i] for i in tmp_dict.keys()}
            tmp_dict = {i: tmp_dict[i].lower() for i in tmp_dict.keys()}
            tmp_dict = {i: re.sub("^\s+|\n|\r|\s+$",' ',tmp_dict[i]) for i in tmp_dict.keys()}
            new_dict.update(tmp_dict)
            
        new_dict['pat_id']=list(set(df_tmp['PCODE'].astype(str).tolist()))[0]
        new_dict['doc_id']=list(set(df_tmp['DCODE'].astype(str).tolist()))[0]
        new_dict['AGE']=list(set(df_tmp['AGE'].astype(str).tolist()))[0]
        new_dict['AGE_days']=list(set(df_tmp['AGE_days'].astype(str).tolist()))[0]
        new_dict['GENDER']=list(set(df_tmp['GENDER'].astype(str).tolist()))[0]
        new_dict['TREATDATE']=list(set(df_tmp['TREATDATE'].astype(str).tolist()))[0]
        new_dict['диагноз по мкб-0']=list(set(df_tmp['DGNAME'].astype(str).tolist()))[0]
        new_dict['шифр диагноза по мкб-0']=list(set(df_tmp['MKBCODE'].astype(str).tolist()))[0],
        new_dict['emr_id']=list(set(df_tmp['PROTOCOLID'].astype(str).tolist()))[0]
        new_dict['doctor_type_id']="ther"
        
        df = pd.DataFrame(dict([ (k,pd.Series(v)) for k,v in new_dict.items() ]))
        res_list.append(df)
    
    df_init_prots = pd.concat(res_list, sort=False)
    df_init_prots = df_init_prots.fillna('')
    df_res_prots = df_init_prots.reset_index(drop=True)
    return df_res_prots

In [5]:
template='''
    SELECT 
    p.PROFNAME,
    d.dcode,
    d.FULLNAME,
    tp.PCODE, 
    c.pol gender,
    floor((t.treatdate - c.bdate)/365) "AGE",
    floor((t.treatdate - c.bdate)) "AGE_days",
    t.TREATDATE,
    tp.TREATCODE, 
    dg.DGNAME,
    dg.MKBCODE,
    gp.CODEPARAMS, 
    gp.NAMEPARAMS, 
    gp.F25_NAMEPARAMS, 
    pif.VALUETEXT,
    pif.MINNORMVALUE,
    pif.MAXNORMVALUE,
    tp.PROTOCOLID
    FROM TREATPLACE tp
    LEFT JOIN PARAMSINFO pif ON pif.TREATCODE=tp.TREATCODE
    LEFT JOIN TREAT t ON t.TREATCODE=tp.TREATCODE
    LEFT JOIN DIAGNOSIS dg ON dg.DGCODE=t.DGCODE
    LEFT JOIN GROUPSPARAMS gp ON gp.CODEPARAMS=pif.CODEPARAMS
    LEFT JOIN GROUPSPARAMSNORM gpn ON gpn.CODEPARAMS=pif.CODEPARAMS
    LEFT JOIN DOCTOR d ON d.dcode=tp.dcode
    LEFT JOIN CLIENTS c ON c.pcode=tp.pcode
    LEFT JOIN PROFESSION p ON p.profid = d.profid
    WHERE 1=1 AND t.DEPNUM=991722445
    AND t.YEARNUM = {year} AND t.MONTHNUM = {month}
'''
df_ik = conn.query_table(template.format(**par_prot))

In [6]:
df_ik.to_csv('lab_'+ str(par_prot['year']) + '_' + str(par_prot['month']) + '.csv', sep='\t')